This guide combines the new orders API (v2) and existing guides for using the python requests package to get Planet imagery.

* https://planet-platform.readme.io/docs/api-examples

* https://github.com/planetlabs/notebooks/blob/master/jupyter-notebooks/data-api-tutorials/planet_data_api_introduction.ipynb

First we import the modules we need. We use a library called `requests`, which allows one to use HTTP methods directly to get data or send data to an endpoint (the Planet v2 API in this case). A lot of APIs use requests under the hood to manage receiving and sending data to users. Since the v2 API is still being built out, we'll need to use requests directly.

In [16]:
import os
import json
import requests
import shutil

def p(data):
    "helper func to pretty print json output"
    print(json.dumps(data, indent=2))
    
URL = "https://api.planet.com/compute/ops/orders/v2" #found on the v2 docs

session = requests.Session()


Make sure you have set your Planet API key in your ~/.bashrc with `export PL_API_KEY='your api key'`. Then make sure that .bashrc file is sourced with `source ~/.bashrc`. We do this so that this secret API key doesn't accidently get exposed through Github.

In [66]:
PLANET_API_KEY = os.environ['PL_API_KEY'] 
session.auth = (PLANET_API_KEY, "")

This get requests goes to the URL we defined above and gets some data on recent activity with the v2 Orders API.

In [67]:
# Make a GET request to the Planet Data API
res = session.get(URL)
res.json()

Next let's define a simple query and download scenes that match that query. Later we will define filters but we can also download specific ids. This can be useful if you have preselected some scenes that look good in the Planet Explorer GUI: https://www.planet.com/explorer/

In [74]:
request = {  
   "name":"simple order",
   "products":[
      {  
         "item_ids":[  
            "20151119_025740_0c74",
            "20151119_025741_0c74"
         ],
         "item_type":"PSScene4Band",
         "product_bundle":"analytic"
      }
   ]
}

Instead of getting data with a GET request, this POST request sends data to the Planet API, telling them what we want to query. They then send us back a result to our request with some information. Requests to REST APIs (a popular API framework) always have a Status Code and if the request was succesful, a JSON body with data. In general, staus codes starting with 2 mean that something worked or is working. You can see what HTTP status codes mean here: https://en.wikipedia.org/wiki/List_of_HTTP_status_codes

In [78]:
res = session.post(URL, json=request)

Now that we have posted a request we can check the status of the POST request with the id of our `res` object, which is what the POST request returned. Note that we need to retreive the contents of the result with the `.json` method. Calling the `res` object will return the HTTP status code that indicates if our POST request was succesful or not.

In [79]:
res

<Response [202]>

In [80]:
p(res.json())

{
  "_links": {
    "_self": "https://api.planet.com/compute/ops/orders/v2/7b4a8057-311f-4d7c-952c-1f83f36e2bfd"
  },
  "created_on": "2018-10-22T05:31:02.707Z",
  "error_hints": [],
  "id": "7b4a8057-311f-4d7c-952c-1f83f36e2bfd",
  "last_message": "Preparing order",
  "last_modified": "2018-10-22T05:31:02.707Z",
  "name": "simple order",
  "products": [
    {
      "item_ids": [
        "20151119_025740_0c74",
        "20151119_025741_0c74"
      ],
      "item_type": "PSScene4Band",
      "product_bundle": "analytic"
    }
  ],
  "state": "initializing"
}


With the JSON output we can check on the status of the order. Looks like our ordered items are ready to download! Since we didn't specify the files we wanted in the request body, for each scene id we get a lot of metadata files along with our analytic .tif

In [82]:
STATUS_URL = res.json()['_links']['_self']
status = session.get(STATUS_URL)
status.json()

{'_links': {'_self': 'https://api.planet.com/compute/ops/orders/v2/7b4a8057-311f-4d7c-952c-1f83f36e2bfd',
  'results': [{'delivery': 'success',
    'expires_at': '2018-10-23T05:31:59.290Z',
    'location': 'https://api.planet.com/compute/ops/download/?token=eyJhbGciOiJIUzUxMiIsInR5cCI6IkpXVCJ9.eyJleHAiOjE1NDAyNzI3MTksInN1YiI6Ilh0cjZyRFBXYVE5aXpXbzdmbUcrNUwzTGg4SWt2dEViRDY4TmQzRXlLc054T0Z0eGNYOXF6TDIyWFZWL0tnVkhTalF4b2tvbEpXWDRweWJLYlR6ZTJnPT0iLCJ0b2tlbl90eXBlIjoiZG93bmxvYWQtYXNzZXQtc3RhY2siLCJhb2kiOiIiLCJhc3NldHMiOlt7Iml0ZW1fdHlwZSI6IiIsImFzc2V0X3R5cGUiOiIiLCJpdGVtX2lkIjoiIn1dLCJ1cmwiOiJodHRwczovL3N0b3JhZ2UuZ29vZ2xlYXBpcy5jb20vY29tcHV0ZS1vcmRlcnMtbGl2ZS83YjRhODA1Ny0zMTFmLTRkN2MtOTUyYy0xZjgzZjM2ZTJiZmQvMS9tYW5pZmVzdC5qc29uP0V4cGlyZXM9MTU0MDI3MjcxOVx1MDAyNkdvb2dsZUFjY2Vzc0lkPWNvbXB1dGUtZ2NzLXN2Y2FjYyU0MHBsYW5ldC1jb21wdXRlLXByb2QuaWFtLmdzZXJ2aWNlYWNjb3VudC5jb21cdTAwMjZTaWduYXR1cmU9QTV1bDBDRW0xbFdHbUM4cnJBSEMlMkJSSk1MVmlVWEhEUUJVSHcxUERjc2JHUDRiWmtuT29FVnZra3hiV0VlUXB3ZjRSV0pxeEVjbW0lMkZvQ

In [64]:
def download_file(url, name, prefix, session):
    '''
    Helper function modified from 
    https://stackoverflow.com/questions/16694907/how-to-download-large-file-in-python-with-requests-py#16696317
    '''
    
    local_filename = os.path.join(prefix, name.split('/')[-1])
    r = session.get(url, stream=True)
    with open(local_filename, 'wb') as f:
        shutil.copyfileobj(r.raw, f)

    return local_filename



In [65]:
for result in status.json()['_links']['results']:
    download_file(result['location'], result['name'], "data", session)

Boom, we have our data.